In [ ]:
pip install datasets transformers wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
import numpy as np
import tensorflow as tf
from transformers import MarianMTModel, MarianTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from IPython.display import Markdown, display
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Sentence pairs in English-German - 2025-02-11.tsv", sep = '\t')

In [ ]:
df.shape

(543741, 4)

In [ ]:
# df = df[0:25000]

In [ ]:
df.head()

,1276,Let's try something.,77,Lass uns etwas versuchen!
0,1276,Let's try something.,392932,Lasst uns etwas ausprobieren!
1,1276,Let's try something.,1674110,Lass uns mal was ausprobieren.
2,1280,Today is June 18th and it is Muiriel's birthday!,81,Heute ist der 18. Juni und das ist der Geburts...
3,1280,Today is June 18th and it is Muiriel's birthday!,1674125,Heute ist der achtzehnte Juni — und Muiriels G...
4,1282,Muiriel is 20 now.,83,Muiriel ist jetzt 20.


In [ ]:
df.columns.values[3] = 'German'

In [ ]:
df.columns.values[1] = 'English'
df.columns = df.columns.str.strip()
print(df.columns)

Index(['1276', 'English', '77', 'German'], dtype='object')


In [ ]:
df.columns

Index(['1276', 'English', '77', 'German'], dtype='object')

In [ ]:
df.head()

,1276,English,77,German
0,1276,Let's try something.,392932,Lasst uns etwas ausprobieren!
1,1276,Let's try something.,1674110,Lass uns mal was ausprobieren.
2,1280,Today is June 18th and it is Muiriel's birthday!,81,Heute ist der 18. Juni und das ist der Geburts...
3,1280,Today is June 18th and it is Muiriel's birthday!,1674125,Heute ist der achtzehnte Juni — und Muiriels G...
4,1282,Muiriel is 20 now.,83,Muiriel ist jetzt 20.


In [ ]:
df.drop(['1276'],axis=1, inplace=True)

In [ ]:
df.drop(['77'],axis=1, inplace=True)

In [ ]:
df

,English,German
0,Let's try something.,Lasst uns etwas ausprobieren!
1,Let's try something.,Lass uns mal was ausprobieren.
2,Today is June 18th and it is Muiriel's birthday!,Heute ist der 18. Juni und das ist der Geburts...
3,Today is June 18th and it is Muiriel's birthday!,Heute ist der achtzehnte Juni — und Muiriels G...
4,Muiriel is 20 now.,Muiriel ist jetzt 20.
...,...,...
543736,The children behaved well.,Die Kinder haben sich gut benommen.
543737,I think that nobody will come.,"Ich glaube, dass niemand kommen wird."
543738,I think nobody will come.,"Ich glaube, dass niemand kommen wird."
543739,The net was full.,Das Netz war voll.


In [ ]:
df['English']=df['English'].apply(lambda x: x.lower())
df['German']=df['German'].apply(lambda x: x.lower())

In [ ]:
df.isna().sum()

,0
English,0
German,0


In [ ]:
from datasets import Dataset


In [ ]:
import pandas as pd
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

def generator():
    for index, row in df.iterrows():
        yield {
            'English': row['English'],
            'German': row['German']
        }

hf_dataset = Dataset.from_generator(generator)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
hf_dataset

Dataset({
    features: ['English', 'German'],
    num_rows: 543741
})

In [ ]:
hf_dataset.save_to_disk('converted_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/543741 [00:00<?, ? examples/s]

In [ ]:
hf_dataset[0]

{'English': "let's try something.", 'German': 'lasst uns etwas ausprobieren!'}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
def tokenize_function(batch):
    inputs = tokenizer(batch['English'], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch['German'], padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": inputs['input_ids'],
        "attention_mask": inputs['attention_mask'],
        "labels": targets['input_ids'],
    }

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/543741 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['English', 'German', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 543741
})

In [ ]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
import wandb

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("Using CPU")
model.to(device)

wandb.init(project='Eng2Ger', name='dhaanusdhaanus2')


training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_translation_model_run',
    eval_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    fp16=True,
    num_train_epochs=3,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],)




Using GPU: NVIDIA A100-SXM4-40GB


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhaanusdhaanus2 (dhaanusdhaanus2-srh-berlin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.092300,0.077924
2,0.074600,0.069570
3,0.067500,0.067335


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=10197, training_loss=0.09354599132203022, metrics={'train_runtime': 2116.8253, 'train_samples_per_second': 616.478, 'train_steps_per_second': 4.817, 'total_flos': 4.423655267028173e+16, 'train_loss': 0.09354599132203022, 'epoch': 3.0})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("./Eng2Ger")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

input_text = '''The father of Niels Bohr, Christian Bohr, was a professor of physiology;[3]his mother Ellen Bohr (née Adler) came from a Jewish family. Together with his father and brother Harald Bohr, he regularly held talks and discussions on scientific topics that strengthened both brothers' interest in the natural sciences and shaped their later lives. “I grew up in a house with a rich intellectual life, where scientific discussions were commonplace. In fact, my father hardly distinguished between his own scientific work and his lively interest in all the problems of human life," Niels Bohr later judges in retrospect of his parental home. Harald Bohr later became professor of mathematics, while Niels Bohr turned to physics. Both were also active in the early days of football on the European continent as football players for the club Akademisk Boldklub, Niels Bohr as goalkeeper. His brother even made the leap into the Danish national team and participated in the first football tournament of the 1908 Summer Olympics. Whether Niels Bohr also came to the honors of a national player is not known due to the source situation of the early Danish international games apart from the Olympic tournaments.'''
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate the translation
translated = model.generate(**inputs)
print("Translation:", tokenizer.decode(translated[0], skip_special_tokens=True))


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translation: der vater von Niels bohr, Christian bohr, war professor für physiologie;[3] seine mutter Ellen bohr (gebirde adler) kam aus einer jüdischen familie. zusammen mit seinem vater und bruder Harald bohr hielt er regelmäßig gespräche und diskussionen zu wissenschaftlichen themen, die das interesse beide brüder an den naturwissenschaften stärkerten und ihr späteres leben formten. „ich bin in einem haus mit einem reichen intellektuellen leben aufgewachsen, in dem wissenschaftliche diskussionen alleinwoll waren. in der tat zwischen seiner eigen wissenschaftlichen arbeit und seinem lebhaften interesse an allen problemen des menschlichen lebens kaum unterscheidete mein vater vater zu unterscheiden. Niels bohr spätiger als als ist als. sein bruder machte sogar in die sprührung in die dänische nationalmannschaft und beim ersten feier der ersten olyten olystag der olydendendenden.


In [ ]:
model.save_pretrained("./Eng2Ger")
tokenizer.save_pretrained("./Eng2Ger")

('./Eng2Ger/tokenizer_config.json',
 './Eng2Ger/special_tokens_map.json',
 './Eng2Ger/vocab.json',
 './Eng2Ger/source.spm',
 './Eng2Ger/target.spm',
 './Eng2Ger/added_tokens.json')

In [ ]:
!zip -r Eng2Ger.zip Eng2Ger

  adding: Eng2Ger/ (stored 0%)
  adding: Eng2Ger/model.safetensors (deflated 7%)
  adding: Eng2Ger/source.spm (deflated 49%)
  adding: Eng2Ger/tokenizer_config.json (deflated 68%)
  adding: Eng2Ger/special_tokens_map.json (deflated 35%)
  adding: Eng2Ger/target.spm (deflated 49%)
  adding: Eng2Ger/generation_config.json (deflated 43%)
  adding: Eng2Ger/config.json (deflated 61%)
  adding: Eng2Ger/vocab.json (deflated 69%)


In [ ]:
from google.colab import files
files.download("Eng2Ger.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>